In [98]:
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
import requests
import pandas as pd

## Web Scraping Historical Team Data

In [94]:
seasons = list(range(1996, 2023))
data_url = "https://www.basketball-reference.com/teams/{}/{}.html"
teams = ["ATL", "BOS", "BRK", "NJN", "CHO", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND",
         "LAC", "LAL", "MEM", "VAN", "MIA", "MIL", "MIN", "NOP", "NOH", "NYK", "OKC", "SEA", "ORL",
         "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS", "WSB", "CHH"]

In [95]:
# instantiating (start) the webdriver
driver = webdriver.Chrome(executable_path="/Users/caleb/Desktop/github/NBA/chromedriver_mac64")

/var/folders/x3/2znzyvg544q8fg1wz_2m7wc80000gn/T/ipykernel_8906/4144818552.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/caleb/Desktop/github/NBA/chromedriver_mac64")


In [96]:
# scrape_season(team) will use the webdriver to scrape the seasons from 1996 to 2022 for each team
def scrape_season(team):
    for season in seasons:
        url = data_url.format(team, season)

        response = requests.get(url)

        # response.status_code returns the HTTP status code received from a server after sending an HTTP request
        # 200:OK, 404: Not Found, 500: Internal Server Error, 429: Too many requests
        if response.status_code == 200:

            # launch a new broswer and opens the given URL in the broswer instance
            driver.get(url)

            # pauses for 3 seconds
            time.sleep(3)
            
            # gets the HTML source code of the page
            page_source = driver.page_source

            # write the contents of the page (page_source) to a file in the DATA directory
            with open("DATA/{}_{}.html".format(team, season), "w+") as file:
                file.write(page_source)
        else:
            # pauses for 3 seconds
            time.sleep(3)
            continue

In [97]:
# scrape every team in the list of teams
for team in teams:
    print(team)
    scrape_season(team)

driver.quit()

LAC
LAL
MEM
VAN
MIA
MIL
MIN
NOP
NOH
NYK
OKC
SEA
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
WSB
CHH


**We have 801 seasons from 30 teams across 27 seasons, 9 missing seasons are 7 from the New Orleans Pelicans/Hornets who didn't exist**<br>
**prior to 2003 and 2 from the Charlotte Hornets/Bobcats who didn't play between 2002-2004**

## Parsing the Data

In [101]:
def parse_data(team):
    for season in seasons:
        file_path = "DATA/{}_{}".format(team, season)

        # checking if file exists
        if os.path.exists(file_path):

            # get the contents of the file containing the team's season data
            with open(file_path) as file:
                team_data = file.read()

            # creates a new object using the BeautifulSoup class and allows for parsing of the team_data
            soup = BeautifulSoup(team_data, "html.parser")

            team_opp_stats = soup.find(id="div_team_and_opponent")

        else:
            continue

FileNotFoundError: [Errno 2] No such file or directory: 'DATA/TOR_1993.html'

In [112]:
file_path = "DATA/{}_{}.html".format("TOR", 2022)


# get the contents of the file containing the team's season data
with open(file_path) as file:
    team_data = file.read()

# creates a new object using the BeautifulSoup class and allows for parsing of the team_data
soup = BeautifulSoup(team_data, "html.parser")

team_opp_stats = soup.find(id="div_team_and_opponent")

team_opp_table = pd.read_html(str(team_opp_stats))[0]

In [114]:
team_opp_table

,Unnamed: 0,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,82.0,19855,3332,7489,0.445,979,2808,0.349,2353,...,0.759,1095,2622,3717,1809,738,376,1021,1607,8970
1,Team/G,NaN,242.1,40.6,91.3,0.445,11.9,34.2,0.349,28.7,...,0.759,13.4,32.0,45.3,22.1,9.0,4.6,12.5,19.6,109.4
2,Lg Rank,NaN,5,14,4,25.000,19,19,20.000,13,...,23.000,2,30,8,29,2,17,2,13,20
3,Year/Year,NaN,0.7%,2.3%,3.0%,-0.003,-17.4%,-12.9%,-0.019,13.6%,...,-0.056,41.4%,-0.5%,9.0%,-8.5%,4.9%,-15.1%,-5.8%,-7.5%,-1.7%
4,Opponent,82.0,19855,3197,6919,0.462,1014,2861,0.354,2183,...,0.789,847,2764,3611,2076,549,416,1298,1563,8782
5,Opponent/G,NaN,242.1,39.0,84.4,0.462,12.4,34.9,0.354,26.6,...,0.789,10.3,33.7,44.0,25.3,6.7,5.1,15.8,19.1,107.1
6,Lg Rank,NaN,5,5,2,17.000,16,15,18.000,5,...,26.000,16,12,14,20,2,25,2,23,7
7,Year/Year,NaN,0.7%,-0.1%,-0.4%,0.001,-11.1%,-4.9%,-0.025,6.0%,...,-0.010,3.3%,-6.0%,-4.0%,-2.6%,1.1%,-9.1%,-1.8%,-2.3%,-4.2%
